In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import re
import traceback
import time



# "                 iPhone13         128GB ブルー             Bランク           SIMフリー"
# "バッテリー100%    iPhone11 Pro      64GB ゴールド           Cランク           SIMフリー"
# "バッテリー100%    iPhone14 Pro Max 256GB ゴールド           Bランク           SIMフリー"
# "バッテリー100%    iPhone15         128GB ブラック           A＋ランク ほぼ新品 SIMフリー"
# "バッテリー90%以上 iPhone11          64GB グリーン            Bランク           SIMフリー"
# "バッテリー100%    iPhone16 Pro Max  1TB ブラックチタニウム   A＋ランク ほぼ新品 SIMフリー"
# "                 iPhoneSE2                                                  SIMフリー バッテリー新品"
# "【未使用】iPhone15 Pro Max 1TB ブラックチタニウム SIMフリー"

records =[]
try:
    # Chrome起動
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 10)
 
    driver.get("https://www.newsedphone.com/collections/iphone")  # ←あなたの対象ページに変更してください
    
    # ページング
    pgae_path = "//div[@class='pagination-wrapper']//li[last()-1]"
    page_count = driver.find_element(By.XPATH, pgae_path)
    
    print(page_count.text)
    
    for i in range(1, int(page_count.text)+1):
        # 各ページ
      iphone_list_path = "//*[@id='product-grid']/li"
      iphone_list = driver.find_elements(By.XPATH, iphone_list_path)
      print(f"{i}ページ目")
      print("iPhoneの個数：", len(iphone_list))

      
      for iphone in iphone_list:
        info = iphone.text
        #"iPhoneSE2 SIMフリー バッテリー新品"のようなカードはスキップ
        if not (("GB" in info or "TB" in info) and "ランク" in info):
          continue
        device = info.split("\n")[0]
        
        
        # "バッテリー90%以上 iPhone6S 16GB シルバー Bランク SIMフリー" のような文字列から　"iPhone6S 16GB シルバー Bランク" のみを取り出す
        # 開始インデックス（"iPhone"の位置）
        start = device.find("iPhone")
        # 終了インデックス（"ランク"の位置 + 「ランク」の長さ）
        end = device.find("ランク") + len("ランク")
        # 部分文字列を抽出
        result = device[start:end]

        # 空白で分割
        parts = result.strip().split()

        # 後ろから3つを取り出し
        memory = parts[-3]
        color = parts[-2]
        grade = parts[-1].replace("ランク", "")

        # 残りをデバイス名として結合
        marketing_name = ' '.join(parts[:-3])

        # print("Device:", device)
        # print("Memory:", memory)
        # print("Grade:", grade)

        price = info.split("\n")[2]
        
        # 数字とカンマだけを取り出す
        price_text = re.search(r'[\d,]+', price).group().replace(',', '')
        
        records.append({
          "Marketing_name": marketing_name,
          "Memory": memory,
          "Grade": grade,
          "Price":int(price_text)
        })
        
      next_page_button = driver.find_element(By.XPATH, "//div[@class='pagination-wrapper']//li[last()]")
      next_page_button.click()
      time.sleep(3)
      driver.execute_script("window.scrollTo(0, 0);")
          
          
      df = pd.DataFrame(records)     
except Exception as e:
    print("処理中にエラーが発生:", e)
    traceback.print_exc()  # 詳細なエラーログ（ファイル名・行番号・スタックトレース）を表示
    print(device)
    print(result)
    

finally:
    driver.quit()
    print("ブラウザを終了しました")

20
1ページ目
iPhoneの個数： 16
2ページ目
iPhoneの個数： 16
3ページ目
iPhoneの個数： 16
4ページ目
iPhoneの個数： 16
5ページ目
iPhoneの個数： 16
6ページ目
iPhoneの個数： 16
7ページ目
iPhoneの個数： 16
8ページ目
iPhoneの個数： 16
9ページ目
iPhoneの個数： 16
10ページ目
iPhoneの個数： 16
11ページ目
iPhoneの個数： 16
12ページ目
iPhoneの個数： 16
13ページ目
iPhoneの個数： 16
14ページ目
iPhoneの個数： 16
15ページ目
iPhoneの個数： 16
16ページ目
iPhoneの個数： 16
17ページ目
iPhoneの個数： 16
18ページ目
iPhoneの個数： 16
19ページ目
iPhoneの個数： 16
20ページ目
iPhoneの個数： 15
ブラウザを終了しました


In [81]:
df.sort_values(by="Price", ascending=False)

,Marketing_name,Memory,Grade,Price
306,iPhone16 Pro Max,1TB,A＋,245000
305,iPhone16 Pro Max,512GB,A＋,228000
304,iPhone16 Plus,256GB,A＋,154000
303,iPhone14 Pro Max,512GB,B,149000
302,iPhone14 Pro Max,512GB,B,149000
...,...,...,...,...
4,iPhone7,32GB,B,8500
2,iPhone6S,32GB,C,8000
1,iPhone6S,16GB,B,8000
3,iPhone6S Plus,16GB,B,8000


In [83]:
df["Memory"].value_counts()

Memory
64GB     117
128GB     94
256GB     50
32GB      28
512GB     12
16GB       5
1TB        1
Name: count, dtype: int64

In [82]:
df.to_csv("newsed_phone.csv")